## Prueba de Conocimiento: Machine Learning & NLP

Usando el dataset de **`Tweets.csv`** y utilizando métodos de procesamiento de datos de **`NLP`**, desarrolla un modelo de predicción sobre la columna de **`sentiment`**.

- Usa diferentes modelos de clasificación y compara sus métricas y el tiempo de ejecución de cada uno.
- Retorna un **`DataFrame`** con los resultados (metricas) de todos los modelos.
- Selecciona el mejor modelo y aplica **`GridSearch()`** para encontrar los mejores parámetros.
- Usa algoritmos de **`PCA`** o de **`SMOTE`** si consideras que es necesario.

In [1]:
import numpy as np
import pandas as pd

# Expresiones regulares
import re

# Procesado de textos
import nltk
# nltk.download()
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

# Modelo Bag-of-Word
from sklearn.feature_extraction.text import CountVectorizer
# Modelo TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer

from nltk.sentiment.vader import SentimentIntensityAnalyzer

# GridSearchCV
from sklearn.model_selection import GridSearchCV

# Train, Test
from sklearn.model_selection import train_test_split

# Metricas
from sklearn.metrics import jaccard_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

# Clasificadores
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Tiempo de ejecución
import time

#GridSearchCV
from sklearn.model_selection import GridSearchCV

In [2]:
df = pd.read_csv("../input/tweets/Tweets.csv")

df.head(3)

In [3]:
df.shape

In [4]:
df.info()

In [5]:
####################
# Preprocesamiento #
####################

In [6]:
# Comprobar si hay NaN en alguna columna
df.isnull().sum()

In [7]:
# Eliminar NaN: hay uno en la columna "text" y en la columna "selected_text"
df.dropna(inplace = True)
df.reset_index(drop = True, inplace = True)

In [8]:
# Eliminar la columna "selected_text", porque no es necesaria
df.drop(["textID", "selected_text"], axis = 1, inplace = True)
df.head(3)

In [9]:
# Pasar los valores de la columna "sentiment" a numéricos
df["sentiment"].value_counts()

In [10]:
sent_dict = {valor : num for num, valor in enumerate(df["sentiment"].unique())}
sent_dict

In [11]:
df["sentiment"].replace(sent_dict, inplace = True)
df["sentiment"].value_counts()

In [12]:
# Preparación de stopwords en inglés
def preparar_stopwords():
    # Cargar las stopwords de Inglés que vienen por defecto
    stopwords = nltk.corpus.stopwords.words("english")
    
    # Añadir palabras detectadas en la revisión a la lista de stopwords 
    stopwords.append("and")
    stopwords.append("http")
    stopwords.append("sooo")
    stopwords.append("soooo")
    
    return stopwords

In [13]:
def limpiar_texto(lista, stopwords):
    
    # Lista para guardar los tokens del texto
    tokens_texto = list()

    # Recorrer la lista de textos
    for texto in lista:
        
        # Lista vacía para recopilar los tokens limpios tras tratarlos
        tokens_limpios = list()
                
        # Tokenizar el texto tras pasarlo a minúsculas y asignarles su tag
        tokens = nltk.word_tokenize(text = texto.lower(), language = "english")
        
        for token in tokens:
            # Si el token no está entre las stopwords, su longitud es mayor que 2 y no contiene dígitos ni apóstrofe
            if ((token not in stopwords) and (len(token) > 2) and (re.findall(r"[\d]", token) == []) and
                not(token.startswith("*")) and (re.findall(r"`", token) == []) and not(token.startswith("//")) and
                (re.findall(r"[_]", token) == []) and not(token.startswith(".")) and
                (re.search(r"\b([a]{2})\w*", token) is None) and (re.search(r"([b]{3})\w*", token) is None)):
                # Es un token válido
                tokens_limpios.append(token)
 
        # Acumular la lista de tokens
        tokens_texto.append(tokens_limpios)
    
    return tokens_texto

In [14]:
# Obtener los tokens de cada fila
df["text_tokens"] = limpiar_texto(df["text"], preparar_stopwords())

df.head(3)

In [15]:
# Función para asociar pos a tag de un token para lematizarlo
def nltk_pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

# Función para lematizar tokens
def lematizar_tokens(lista):
    
    # Lista para guardar los tokens lematizados
    lem_texto = list()

    # Inicializar el lematizador
    lemmatizer = WordNetLemmatizer()
    
    # Recorrer la lista de listas de tokens
    for num, tokens in enumerate(lista):

        # Lista vacía para guardar la lematización de la lista de tokens
        tokens_lem = list()

        # Si hay tokens para lematizar
        if len(tokens) > 0:

            # Obtener el tag de los tokens de cada fila
            tokens_tags = nltk.pos_tag(tokens)

            # Por cada token y tag
            for token, tag in tokens_tags:
                # Obtener la pos asociada al tag
                pos = nltk_pos_tagger(tag)
                # Si el tag no tiene pos asociado
                if pos is None:
                    # Guardar el token
                    tokens_lem.append(token)
                else:        
                    # Lematizar el token
                    tokens_lem.append(lemmatizer.lemmatize(token, pos))

        # Acumular la lista de lematización de tokens
        lem_texto.append(" ".join(tokens_lem))

    return lem_texto

In [16]:
# Lematizar los tokens de cada fila
df["text_tokens_lem"] = lematizar_tokens(df["text_tokens"])

df.head(3)

In [17]:
df.shape[0]

In [18]:
############################################################
# Preparar datos para aplicar los modelos de clasificación #
############################################################

In [19]:
def feature_extraction(texto_lematizado):
    
    # Bag-of-Word
    count_vectorizer = CountVectorizer()

    frases = np.array(texto_lematizado)

    # Entrenamos el modelo y transformamos los datos.
    bag = count_vectorizer.fit_transform(frases)

    # Obtener la lista de vocabulario que actúa como cabecera de las columnas
    vocabulario_ordenado = sorted(count_vectorizer.vocabulary_, key = lambda x : count_vectorizer.vocabulary_[x])
    
    # TF-IDF

    # Inicializamos un objeto Tfidf
    tfidf = TfidfTransformer()

    # Cambio la precisión de python a 2 decimales
    np.set_printoptions(precision = 2)

    # Entrenamos el Tfidf y transformamos la variable bag
    bag = tfidf.fit_transform(bag).toarray()

    bag = pd.DataFrame(data = bag, columns = vocabulario_ordenado)

    return bag

In [20]:
bag = feature_extraction(df["text_tokens_lem"])
bag.shape

In [21]:
X_train, X_test, y_train, y_test = train_test_split(bag, df["sentiment"], test_size = 0.3, random_state = 42)

print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_test: {X_test.shape},  y_test: {y_test.shape}")

In [22]:
#################################################################################################
# Ejecutar diferentes modelos de clasificación y comparar sus métricas y el tiempo de ejecución #
#################################################################################################

In [23]:
# Función para aplicar diferentes modelos de clasificación y guardar las métricas
def metricas_modelos_clasificacion(lista_clasificadores, X_train, y_train, X_test, y_test):
    
    # Inicializar el DataFrame resultado y las listas que lo compondrán
    df_clasificadores = pd.DataFrame()
    lista_clasificador = list()
    lista_jaccard_index = list()
    lista_accuracy = list()
    lista_precision = list()
    lista_sensibilidad = list()
    lista_f1_score = list()
    lista_confusion_matrix = list()
    lista_tiempo = list()

    # Por cada método de la lista de clasificadores
    for clasificador in lista_clasificadores:
        
        print(str(clasificador))

        # Momento de inicio de ejecución del modelo
        inicio = time.time()

        # Entrenar el método
        clasificador.fit(X_train, y_train)

        # Calcular la predicción
        yhat = clasificador.predict(X_test)

        # Momento de fin de ejecución del modelo
        fin = time.time()
        
        tiempo = fin - inicio

        # Calcular las métricas
        jaccard_m = jaccard_score(y_test, yhat, average = "macro")
        accuracy_m = accuracy_score(y_test, yhat)
        precision_m = precision_score(y_test, yhat, average = "macro")
        recall_m = recall_score(y_test, yhat, average = "macro")
        f1_m = f1_score(y_test, yhat, average = "macro")
        conf_matrix = confusion_matrix(y_test, yhat)

        print("Jaccard Index:", jaccard_m)
        print("Accuracy:"     , accuracy_m)
        print("Precisión:"    , precision_m)
        print("Sensibilidad:" , recall_m)
        print("F1-score:"     , f1_m)
        print("Confusion Matrix:\n", conf_matrix)
        print(f"Tiempo de ejecución: {tiempo}")
        print("*"*100)
        
        # Guardar el nombre del método de clasificación
        lista_clasificador.append(str(clasificador))
        # Guardar las métricas
        lista_jaccard_index.append(jaccard_m)
        lista_accuracy.append(accuracy_m)
        lista_precision.append(precision_m)
        lista_sensibilidad.append(recall_m)
        lista_f1_score.append(f1_m)
        lista_confusion_matrix.append(conf_matrix)
        # Guardar el tiempo de ejecución
        lista_tiempo.append(tiempo)
    
    # Guardar las listas en el DataFrame
    df_clasificadores["metodo_clasificador"] = lista_clasificador
    df_clasificadores["jaccard_index"] = lista_jaccard_index
    df_clasificadores["accuracy"] = lista_accuracy
    df_clasificadores["precision"] = lista_precision
    df_clasificadores["sensibilidad"] = lista_sensibilidad
    df_clasificadores["f1_score"] = lista_f1_score
    df_clasificadores["confusion_matrix"] = lista_confusion_matrix
    df_clasificadores["tiempo_ejecucion"] = lista_tiempo

    return df_clasificadores

In [24]:
lista_clasificadores = [KNeighborsClassifier(n_neighbors = 3),
                        RadiusNeighborsClassifier(radius = 0.8,
                                                  outlier_label = "most_frequent"),
                        NearestCentroid(metric = "euclidean"),
                        GaussianNB(),
                        LogisticRegression(),
                        DecisionTreeClassifier(),
                        RandomForestClassifier()
                       ]

In [25]:
%%time

metricas_modelos_clasificacion(lista_clasificadores, X_train, y_train, X_test, y_test)

In [26]:
# El modelo más adecuado en base a las métricas es RandomForestClassifier

In [27]:
############################################################################################
# Seleccionar el mejor modelo y aplicar GridSearch() para encontrar los mejores parámetros #
############################################################################################

In [28]:
def aplicar_GridSearchCV(modelo, params, scorers, metric):
    
    model = modelo
    
    grid_solver = GridSearchCV(estimator  = model    , 
                           param_grid = params   , 
                           scoring    = scorers  ,
                           cv         = 5        ,
                           refit      = metric,
                           n_jobs     = -1        )
    model_result = grid_solver.fit(X_train, y_train)

    print(model_result.cv_results_["mean_test_recall_macro"].mean())
    print(model_result.cv_results_["mean_test_f1_macro"].mean())
    print(model_result.cv_results_["mean_test_accuracy"].mean())

    print("*"*100)

    print(model_result.best_score_)
    print(model_result.best_params_)

In [ ]:
modelo = RandomForestClassifier()

params = {"n_estimators"           : [100, 200, 300], # Numero de arboles
          "criterion"              : ["gini", "entropy"], # Función para medir la calidad de una división/split
          "max_depth"              : [3, 4, 5], # Profundidad máxima del árbol
          "max_features"           : [2, 3], # Número de características a considerar en cada split
          "max_leaf_nodes"         : [8], # Máximo número de nodos hoja del árbol
          "min_impurity_decrease"  : [0.02, 0.3], # Un nodo se dividirá si esta división induce una disminución de la impureza mayor o igual a este valor.
          "min_samples_split"      : [2, 5]} # Número mínimo de muestras requeridas para llegar a nodo hoja

scorers = {"f1_macro", "accuracy", "recall_macro"}

metric = "accuracy"

aplicar_GridSearchCV(modelo, params, scorers, metric)


In [ ]:
# Ejecución del resto de métodos de clasificación que requieren más tiempo

In [ ]:
%%time

lista_clasificadores2 = [AdaBoostClassifier(),
                         GradientBoostingClassifier(),
                         SVC()]


metricas_modelos_clasificacion(lista_clasificadores2, X_train, y_train, X_test, y_test)
